In [4]:
import matplotlib.pyplot as plt
import matplotlib
import geopandas as gpd
import xarray as xr
import cartopy.crs as ccrs
import warnings
import pandas as pd

In [2]:
#get the IPBES sub regions
# Read the shapefile
shp_dir = "/storage/workspaces/wa_climate/climate_trt/chari/IPBES_subregions/IPBES_Regions_Subregions2.shp"
shp = gpd.read_file(shp_dir)

# Define the region mapping
region_mapping = {
    'Caribbean': 'North America',  # Corrected typo in 'Caribbean'
    'Central Africa': 'Sub-saharan Africa',
    'Central and Western Europe': 'Central and Western Europe',
    'Central Asia': 'Central Asia',
    'East Africa and adjacent islands': 'Sub-saharan Africa',
    'Eastern Europe': 'Eastern Europe',
    'Mesoamerica': 'North America',
    'North Africa': 'North Africa',
    'North America': 'North America',
    'North-East Asia': 'North-East Asia',  # Corrected typo in 'Nort-East Asia'
    'Oceania': 'Oceania',
    'South America': 'South America',
    'South Asia': 'South Asia',
    'South-East Asia': 'South-East Asia',
    'Southern Africa': 'Sub-saharan Africa',
    'West Africa': 'Sub-saharan Africa',
    'Western Asia': 'Western Asia'
}

# Create the new_region column in the shapefile
shp['new_region'] = shp['Sub_Region'].map(region_mapping)

#

In [ ]:
#read in newvalue_future and _historical and sumbin_future and _historical and plot

warnings.filterwarnings('ignore', category=UserWarning)


scenarios = ["rcp26","rcp60"]
historical_time= 1146
future_time=65

year_indices = {1146: '1995', 35: '2050', 65: '2080', 85: '2100'}

for row, scenario in enumerate(scenarios):

    newvalue_future = xr.open_dataset(f"/storage/scratch/users/ch21o450/data/SR/SR_CC_{future_time}_{scenario}.nc", decode_times=False).to_array().isel(variable=0)
    sumbin_future = xr.open_dataset(f"/storage/scratch/users/ch21o450/data/SR/SR_CCLUC_{future_time}_{scenario}.nc", decode_times=False).to_array().isel(time=0)
    sumbin_future= sumbin_future.isel(variable=0)

    newvalue_hist = xr.open_dataset(f"/storage/scratch/users/ch21o450/data/SR/SR_CC_{historical_time}_{scenario}.nc", decode_times=False).to_array().isel(variable=0)
    sumbin_hist = xr.open_dataset(f"/storage/scratch/users/ch21o450/data/SR/SR_CCLUC_{historical_time}_{scenario}.nc", decode_times=False).to_array().isel(variable=0)

    diff_newvalue = newvalue_future - newvalue_hist
    diff_sumbin = sumbin_future - sumbin_hist


In [6]:
# Create an empty dictionary to store mean species richness for each region and each scenario
mean_species_richness_dict = {'newvalue_future': {}, 'sumbin_future': {}, 'newvalue_hist': {}, 'sumbin_hist': {}, 'diff_newvalue': {}, 'diff_sumbin': {}}

scenarios = ["rcp26","rcp60"]
historical_time= 1146
future_time=65

# Iterate through unique new regions
for new_region in shp['new_region'].unique():
    # Select the region-specific geometry from the shapefile
    region_gdf = shp[shp['new_region'] == new_region]

    for scenario in scenarios:
        # Open datasets for the current scenario
        newvalue_future = xr.open_dataset(f"/storage/scratch/users/ch21o450/data/SR/SR_CC_{future_time}_{scenario}.nc", decode_times=False).to_array().isel(variable=0)
        sumbin_future = xr.open_dataset(f"/storage/scratch/users/ch21o450/data/SR/SR_CCLUC_{future_time}_{scenario}.nc", decode_times=False).to_array().isel(time=0)
        sumbin_future = sumbin_future.isel(variable=0)

        newvalue_hist = xr.open_dataset(f"/storage/scratch/users/ch21o450/data/SR/SR_CC_{historical_time}_{scenario}.nc", decode_times=False).to_array().isel(variable=0)
        sumbin_hist = xr.open_dataset(f"/storage/scratch/users/ch21o450/data/SR/SR_CCLUC_{historical_time}_{scenario}.nc", decode_times=False).to_array().isel(variable=0)

        # Calculate differences
        diff_newvalue = newvalue_future - newvalue_hist
        diff_sumbin = sumbin_future - sumbin_hist

        # Extract the bounding box coordinates
        minx, miny, maxx, maxy = region_gdf.geometry.total_bounds

        # Create a boolean mask based on the bounding box
        mask = (
            (newvalue_future.lon >= minx) & (newvalue_future.lon <= maxx) &
            (newvalue_future.lat >= miny) & (newvalue_future.lat <= maxy)
        )

        # Apply the mask to the DataArrays
        region_data = {
            'newvalue_future': newvalue_future.where(mask),
            'sumbin_future': sumbin_future.where(mask),
            'newvalue_hist': newvalue_hist.where(mask),
            'sumbin_hist': sumbin_hist.where(mask),
            'diff_newvalue': diff_newvalue.where(mask),
            'diff_sumbin': diff_sumbin.where(mask)
        }

        # Calculate mean species richness for each variable
        for variable, data_array in region_data.items():
            mean_species_richness = data_array.mean(dim=['lat', 'lon']).item()
            mean_species_richness_dict[variable].setdefault(new_region, {})[scenario] = mean_species_richness



# Print the tables
for variable, region_data_dict in mean_species_richness_dict.items():
    print(f"\n{variable} Mean Species Richness:")
    result_df = pd.DataFrame(region_data_dict).transpose()
    print(result_df)


newvalue_future Mean Species Richness:
                                 rcp26       rcp60
North America               394.934488  397.175412
South Asia                  387.111492  386.728109
Sub-saharan Africa          508.186747  496.245690
Central and Western Europe  450.271596  449.183801
Western Asia                330.540541  322.629775
South America               628.845941  609.216409
Eastern Europe              376.868086  384.961614
Oceania                     521.211564  511.133301
NaN                                NaN         NaN
South-East Asia             585.382585  581.798113
North-East Asia             433.510105  430.467999
North Africa                333.812781  326.938990
Central Asia                366.053535  355.595902

sumbin_future Mean Species Richness:
                                 rcp26       rcp60
North America               103.365934  100.056678
South Asia                  104.104767   96.986955
Sub-saharan Africa          125.320409   86.669351
Cent

In [7]:
# Create an empty dictionary to store mean species richness for each region and each scenario
mean_species_richness_dict = {'newvalue_future': {}, 'sumbin_future': {}, 'newvalue_hist': {}, 'sumbin_hist': {}, 'diff_newvalue': {}, 'diff_sumbin': {}}

scenarios = ["rcp26","rcp60"]
historical_time= 1146
future_time=65

# Iterate through unique new regions
for new_region in shp['new_region'].unique():
    # Select the region-specific geometry from the shapefile
    region_gdf = shp[shp['new_region'] == new_region]

    for scenario in scenarios:
        # Open datasets for the current scenario
        newvalue_future = xr.open_dataset(f"/storage/scratch/users/ch21o450/data/SR/SR_CC_{future_time}_{scenario}.nc", decode_times=False).to_array().isel(variable=0)
        sumbin_future = xr.open_dataset(f"/storage/scratch/users/ch21o450/data/SR/SR_CCLUC_{future_time}_{scenario}.nc", decode_times=False).to_array().isel(time=0)
        sumbin_future = sumbin_future.isel(variable=0)

        newvalue_hist = xr.open_dataset(f"/storage/scratch/users/ch21o450/data/SR/SR_CC_{historical_time}_{scenario}.nc", decode_times=False).to_array().isel(variable=0)
        sumbin_hist = xr.open_dataset(f"/storage/scratch/users/ch21o450/data/SR/SR_CCLUC_{historical_time}_{scenario}.nc", decode_times=False).to_array().isel(variable=0)

        # Calculate differences
        diff_newvalue = newvalue_future - newvalue_hist
        diff_sumbin = sumbin_future - sumbin_hist

        # Extract the bounding box coordinates
        minx, miny, maxx, maxy = region_gdf.geometry.total_bounds

        # Create a boolean mask based on the bounding box
        mask = (
            (newvalue_future.lon >= minx) & (newvalue_future.lon <= maxx) &
            (newvalue_future.lat >= miny) & (newvalue_future.lat <= maxy)
        )

        # Apply the mask to the DataArrays
        region_data = {
            'newvalue_future': newvalue_future.where(mask),
            'sumbin_future': sumbin_future.where(mask),
            'newvalue_hist': newvalue_hist.where(mask),
            'sumbin_hist': sumbin_hist.where(mask),
            'diff_newvalue': diff_newvalue.where(mask),
            'diff_sumbin': diff_sumbin.where(mask)
        }

        # Calculate mean species richness for each variable
        for variable, data_array in region_data.items():
            mean_species_richness = data_array.sum(dim=['lat', 'lon']).item()
            mean_species_richness_dict[variable].setdefault(new_region, {})[scenario] = mean_species_richness



# Print the tables
for variable, region_data_dict in mean_species_richness_dict.items():
    print(f"\n{variable} Mean Species Richness:")
    result_df = pd.DataFrame(region_data_dict).transpose()
    print(result_df)


newvalue_future Mean Species Richness:
                                   rcp26         rcp60
North America               1.886721e+07  1.897426e+07
South Asia                  1.816327e+06  1.814528e+06
Sub-saharan Africa          4.728169e+06  4.617070e+06
Central and Western Europe  3.148749e+06  3.141142e+06
Western Asia                6.838884e+05  6.675210e+05
South America               4.491218e+06  4.351024e+06
Eastern Europe              1.186004e+07  1.211474e+07
Oceania                     1.239806e+07  1.215833e+07
NaN                         0.000000e+00  0.000000e+00
South-East Asia             1.285500e+06  1.277629e+06
North-East Asia             3.247424e+06  3.224636e+06
North Africa                1.808931e+06  1.771682e+06
Central Asia                1.162220e+06  1.129017e+06

sumbin_future Mean Species Richness:
                                   rcp26         rcp60
North America               1.157781e+07  1.120715e+07
South Asia                  1.058225e+06  

In [9]:

abs_species_richness_dict = {'newvalue_future': {}, 'sumbin_future': {}, 'newvalue_hist': {}, 'sumbin_hist': {}, 'diff_newvalue': {}, 'diff_sumbin': {}}

# Iterate through unique new regions
for new_region in shp['new_region'].unique():
    # Select the region-specific geometry from the shapefile
    region_gdf = shp[shp['new_region'] == new_region]

    for scenario in scenarios:
        # Open datasets for the current scenario
        newvalue_future = xr.open_dataset(f"/storage/scratch/users/ch21o450/data/SR/SR_CC_{future_time}_{scenario}.nc", decode_times=False).to_array().isel(variable=0)
        sumbin_future = xr.open_dataset(f"/storage/scratch/users/ch21o450/data/SR/SR_CCLUC_{future_time}_{scenario}.nc", decode_times=False).to_array().isel(time=0)
        sumbin_future = sumbin_future.isel(variable=0)

        newvalue_hist = xr.open_dataset(f"/storage/scratch/users/ch21o450/data/SR/SR_CC_{historical_time}_{scenario}.nc", decode_times=False).to_array().isel(variable=0)
        sumbin_hist = xr.open_dataset(f"/storage/scratch/users/ch21o450/data/SR/SR_CCLUC_{historical_time}_{scenario}.nc", decode_times=False).to_array().isel(variable=0)

        # Calculate differences
        diff_newvalue = newvalue_future - newvalue_hist
        diff_sumbin = sumbin_future - sumbin_hist

        # Extract the bounding box coordinates
        minx, miny, maxx, maxy = region_gdf.geometry.total_bounds

        # Create a boolean mask based on the bounding box
        mask = (
            (newvalue_future.lon >= minx) & (newvalue_future.lon <= maxx) &
            (newvalue_future.lat >= miny) & (newvalue_future.lat <= maxy)
        )

        # Apply the mask to the DataArrays
        region_data = {
            'newvalue_future': newvalue_future.where(mask),
            'sumbin_future': sumbin_future.where(mask),
            'newvalue_hist': newvalue_hist.where(mask),
            'sumbin_hist': sumbin_hist.where(mask),
            'diff_newvalue': diff_newvalue.where(mask),
            'diff_sumbin': diff_sumbin.where(mask)
        }

        # Calculate absolute species richness for each variable
        for variable, data_array in region_data.items():
            abs_species_richness = data_array.sum(dim=['lat', 'lon']).item()
            abs_species_richness_dict[variable].setdefault(new_region, {})[scenario] = abs_species_richness

# Calculate the percentage change
percentage_change_dict = {}
for new_region in shp['new_region'].unique():
    percentage_change_dict.setdefault(new_region, {})

    for scenario in scenarios:
        diff_newvalue = abs_species_richness_dict['diff_newvalue'][new_region][scenario]
        diff_sumbin = abs_species_richness_dict['diff_sumbin'][new_region][scenario]

        # Avoid division by zero
        if diff_newvalue != 0:
            percentage_change = (diff_sumbin - diff_newvalue) / diff_newvalue * 100
        else:
            percentage_change = float('nan')

        percentage_change_dict[new_region][scenario] = percentage_change

# Print the tables
for variable, region_data_dict in abs_species_richness_dict.items():
    print(f"\n{variable} Absolute Species Richness:")
    result_df = pd.DataFrame(region_data_dict).transpose()
    print(result_df)

# Print the percentage change table
print("\nPercentage Change between diff_newvalue and diff_sumbin:")
percentage_change_df = pd.DataFrame(percentage_change_dict).transpose()
print(percentage_change_df)


newvalue_future Absolute Species Richness:
                                   rcp26         rcp60
North America               1.886721e+07  1.897426e+07
South Asia                  1.816327e+06  1.814528e+06
Sub-saharan Africa          4.728169e+06  4.617070e+06
Central and Western Europe  3.148749e+06  3.141142e+06
Western Asia                6.838884e+05  6.675210e+05
South America               4.491218e+06  4.351024e+06
Eastern Europe              1.186004e+07  1.211474e+07
Oceania                     1.239806e+07  1.215833e+07
NaN                         0.000000e+00  0.000000e+00
South-East Asia             1.285500e+06  1.277629e+06
North-East Asia             3.247424e+06  3.224636e+06
North Africa                1.808931e+06  1.771682e+06
Central Asia                1.162220e+06  1.129017e+06

sumbin_future Absolute Species Richness:
                                   rcp26         rcp60
North America               1.157781e+07  1.120715e+07
South Asia                  1.0582

In [ ]:
Percentage Change=( 
Old Value
New Value−Old Value
​
 )×100